# Preprocess Dataset

In [ ]:
import pandas as pd
import numpy as np
import os, re
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
def load_msss_tables(msss_tables_dir : str) -> dict:
    MSSS_TABLES = [
        "AD_Outage_Type.xls",
        "AD_Componant.xls",
        "AD_Object.xls",
        "AD_Damage.xls",
        "AD_Cause.xls"
    ]

    msss_tables = []

    for level, table in enumerate(MSSS_TABLES):
        path = os.path.join(msss_tables_dir, table)
        data = pd.read_excel(path, header=1, index_col="Code")
        data = data.dropna(axis=1, how='all') # Drop null rows

        # Get the column which contains the MSSS Code
        msss_code_column = "Aux2" if "Filter_Key" in data else "Aux1"
        # Rename this column to "ID"
        data = data.rename(columns={"Description": "Name", msss_code_column: "ID"})

        # If "ID" column only has two states, convert it into a boolean
        if data["ID"].nunique() == 1:
            value = data["ID"].unique()[0]
            data["ID"] = data["ID"].map(lambda x: x == value)

        # Get the column which contains the previous MSSS code
        if "Filter_Key" in data:
            # Rename this column to "PrevID"
            data = data.rename(columns={"Filter_Key":"PrevID"})
        else:
            # If none exists, create an empty one
            data.insert(data.columns.size, "PrevID", np.nan)

        data = data[["Name", "ID", "PrevID"]]
        msss_tables.append(data)

    return msss_tables

In [ ]:
def _get_codes(table : pd.DataFrame, prev_codes : list | None = None, prev_table : pd.DataFrame | None = None) -> list:
    """This function generates a list of MSSS codes given a MSSS table.
    It can be used recursively to generate every unique combination of MSSS codes in the MSSS taxonomy.

    Args:
        table (DataFrame): A MSSS table from load_msss_tables().
        prev_codes (list, optional): List of the MSSS codes from the previous table.
        prev_table (pd.DataFrame, optional): The previous MSSS table. Used to translate MSSS codes into indices.

    Returns:
        codes (list): A list of the table's MSSS codes.
    """
    
    if not prev_codes:
        # If the table does not have a previous MSSS Code column,
        # we can simply use the unique MSSS Codes from the table
        if "PrevID" not in table.dropna(axis=1,how='any'):
            codes = table["ID"].unique().tolist()
            return codes
        else:
            raise TypeError("Getting MSSS codes for this table requires the codes from the previous table!")
    
    codes = prev_codes

    # Use the MSSS Code column as the category
    if not pd.api.types.is_bool_dtype(table["ID"]):
        category_column = "ID"

    # Unless the column is a boolean, in which case use the numerical code
    else:
        table = table.reset_index()
        category_column = "Code"

    # For each category in the previous MSSS table
    for i, code in enumerate(prev_codes):

        last_code = code[-1] if type(code) is list else code

        # Get all sub-categories in this MSSS table
        subcodes = table[table["PrevID"] == last_code][category_column].unique().tolist()
        
        # Convert the last code entry from MSSS Code -> numerical code
        if prev_table is not None:
            last_code_index = prev_table[prev_table["ID"] == last_code].index[0]

            if type(code) is list:
                code[-1] = last_code_index
            else: code = last_code_index

        if type(code) is list:
            subcodes = [[*code, subcode] for subcode in subcodes]
        else:
            subcodes = [[code, subcode] for subcode in subcodes]
        
        codes[i] = subcodes

    # Flatten the list
    codes = [i for j in codes for i in j]

    return codes

In [ ]:
def create_msss_lookup_table(tables : list[pd.DataFrame]) -> pd.DataFrame:
    """
    Create a lookup table with a row for every unique MSSS classification.

    Args:
        tables (list[DataFrame]): List of pre-processed MSSS tables obtained with load_msss_tables().

    Returns:
        DataFrame: The MSSS lookup table.
    """

    # Create a list of lists for every possible MSSS classification
    codes = None
    for table_id, table in enumerate(tables):
        codes = _get_codes(table, codes, tables[table_id - 1] if table_id > 0 else None)
    
    # Create a DataFrame to represent all the MSSS classifications
    msss_features = ["Outage Type", "Component", "Object", "Damage", "Cause"]
    index = pd.DataFrame(codes, columns=msss_features)

    # Add features in the index for the Description and MSSS Code for each feature
    for column, table in zip(msss_features, tables):
        index[f"{column} Description"] = index[column].map(lambda x: table["Name"][x])
        index[f"{column} Code"] = index[column].map(lambda x: table["ID"][x])
    
    return index

In [ ]:
msss_tables_path = "../data/eq/datasets/ergon-ffa-msss-tables"
tables = load_msss_tables(msss_tables_path)

In [ ]:
msss_table = create_msss_lookup_table(tables)

In [ ]:
msss_table.to_csv("data/eq/datasets/msss_table.csv", index=False)

In [ ]:
def get_classif(msss_object_description : str, msss_damage_description : str, msss_cause_description : str, lookup_table : pd.DataFrame) -> pd.DataFrame:
    t = lookup_table
    return t.loc[
        (t["Object Description"] == msss_object_description) & 
        (t["Damage Description"] == msss_damage_description) &
        (t["Cause Description"] == msss_cause_description)
    ]

In [ ]:
EQ_DATA_PATH = "../data/eq/datasets/2024-tim-elkins-failures-data/"

EQ_DATA_FILES = [
    "EE Failures FY23-24 (Updated).xlsx",
    "EGX Failures FY23-24 (updated).xlsx"
]

EQ_DATA_PROVIDERS = ["Ergon Energy", "Energex"]

EQ_DATA_FILES = [os.path.join(EQ_DATA_PATH, file) for file in EQ_DATA_FILES]

In [ ]:
# Load all the datasets
data = [pd.read_excel(dataset) for dataset in EQ_DATA_FILES]

In [ ]:
index = "OUTAGE_ID"

input_features = [
"WEATHER_CONDITION",
"OUTAGE_CAUSE",
"FAULT_LONG_DESCRIPTION",
"SHORT_DESC_2",
"WORK_ORDER_COMPONENT_CODE_DESCRIPTION",
"OUTAGE_CAUSE_GROUP",
"OUTAGE_STANDARD_REASON_DESCRIPTION",
"REASON_FOR_INTERRUPTION",
"PROVIDER" # We add this feature to the datasets during pre-processing
]

output_labels = [
    "MSSS_OBJECT_DESCRIPTION",
    "MSSS_DAMAGE_DESCRIPTION",
    "MSSS_CAUSE_DESCRIPTION"
]

In [ ]:
# Data backup
# Executing this cell will restore the dataset to the original state

from copy import copy
if "data2" in locals():
    data = data2
data2 = copy(data)

In [ ]:
# Capitalize all column names and replace spaces with underscores
for i, dataset in enumerate(data):
    columns = dataset.columns.to_list()
    new_columns = [column.replace(" ", "_").upper() for column in columns]
    replacement = dict(zip(columns, new_columns))
    data[i] = dataset.rename(columns=replacement)

for i, dataset in enumerate(data):
    # Drop all non input/output columns
    features = input_features + output_labels + [index]
    features = [f for f in features if f in dataset.columns]
    dataset = dataset[features]
    
    # Drop all rows with entirely null input features
    features = [f for f in features if f in input_features]
    dataset = dataset.dropna(subset=features, how='all')
    
    # Add a feature to tell which dataset we're using (EE/EGX)
    provider = EQ_DATA_PROVIDERS[i]
    dataset.insert(dataset.columns.size, "PROVIDER", provider)

    data[i] = dataset

In [ ]:
# Certain null-like string values should be replaced with null
for i, dataset in enumerate(data):
    null_like = ["<null>", "#REF!"]

    for j in null_like: dataset = dataset.replace(j, np.nan)
    
    data[i] = dataset

In [ ]:
# Remove all rows with duplicate outage ID and output features
for i, dataset in enumerate(data):
    dataset = dataset.drop_duplicates(subset=[index, *output_labels], keep='last')
    data[i] = dataset

In [ ]:
# Concatenate all datasets into one
dataset = pd.concat(data)

In [ ]:
# Make spacing / capitalisation of MSSS feature labels consistent
dataset[output_labels] = dataset[output_labels].map(lambda x: x.strip().title() if type(x) is str else x)

In [ ]:
# Correct inconsistent capitalisation within MSSS feature labels where possible

def map_item_to_msss_feature_label(item : str, label_names : list[str]) -> str | None:
    """
    Given a MSSS label (e.g., " animal"), try to map
    it to an MSSS label from a list (e.g., "Animal")
    to correct issues in capitalisation / spacing.

    Args:
        item (str): The raw label name.
        label_names (list[str]): List of legal label names.

    Returns:
        label (str | None): The correct label name, if found, otherwise the raw label name.
    """
    if pd.isna(item): return np.nan

    if item in label_names: return item

    original_item = item
    item = item.strip().lower()

    labels_easy = [i.strip().lower() for i in label_names]
    if item in labels_easy:
        index = labels_easy.index(item)
        return label_names[ index ]

    word_matches = [i for i in labels_easy if item in i]
    
    if word_matches:
        if len(word_matches) == 1:
            item = word_matches[0]
            index = labels_easy.index(item)
            return label_names[ index ]
    
    return original_item

def rectify_feature(dataset : pd.DataFrame, msss_feature : str, msss_table : pd.DataFrame) -> pd.DataFrame:
    """Correct an MSSS feature from a dataset by removing inconsistencies in capitalisation / spacing.

    Args:
        dataset (pd.DataFrame): The original dataset
        msss_feature (str): MSSS feature name, e.g., "Cause".
        msss_table (pd.DataFrame): MSSS lookup table.

    Returns:
        pd.DataFrame: The dataset with msss_feature rectified.
    """
    table_column = f"{msss_feature} Description"
    data_column = f"MSSS_{msss_feature.upper()}_DESCRIPTION"

    label_names = msss_table[table_column].unique().tolist()

    dataset[data_column] = dataset[data_column].map(lambda x: map_item_to_msss_feature_label(x, label_names))
    return dataset

for feature in ["Object", "Cause", "Damage"]:
    dataset = rectify_feature(dataset, feature, msss_table)

# Train / Eval Split

In [ ]:
# Shuffle the data in a deterministic way
dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
data_train = dataset.dropna(subset=output_labels, how='any')

In [ ]:
# Supervised dataset
data_train = dataset.dropna(subset=output_labels, how='any')

# Unsupervised dataset
data_eval = dataset.drop(data_train.index)

# Reduce training data class imbalances

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)

In [ ]:
from copy import copy
data_resampled = copy(data_train)
size = len(data_resampled)

In [ ]:
# Remove all classes with less than 50 samples
min_class_size = 50
while True:
    for label in output_labels:
        class_counts = data_resampled[label].value_counts()
        classes_to_remove = class_counts.where(lambda x: x <= min_class_size).dropna().keys().to_list()
        data_resampled = data_resampled[~data_resampled[label].isin(classes_to_remove)]

    if [data_resampled[label].value_counts().where(lambda x: x <= min_class_size).dropna().size for label in output_labels] == [0] * len(output_labels):
        break

In [ ]:
# Oversample minority classes to improve class distribution
for label in output_labels:
    features = input_features + [i for i in output_labels if not i == label]

    X, y = data_resampled[features], data_resampled[label]
    
    X, y = ros.fit_resample(X, y)

    data_resampled = pd.concat([X, y], axis=1)
    data_resampled = data_resampled.dropna()
    data_resampled = data_resampled.sample(n=size)

In [ ]:
# Compare old VS new class distributions

from matplotlib import pyplot as plt

for i, label in enumerate(output_labels):
    ax = plt.subplot(2,3,i + 1)
    plt.plot(data_train[label].value_counts())
    plt.xticks([])
    #plt.ylim([0,10000])
    plt.title(label.rstrip("DESCRIPTION").lstrip("MSSS"))
    ax.set_yscale('log')
    plt.ylim([1, 100000])

plt.suptitle("MSSS class distribution")
for i, label in enumerate(output_labels):
    ax = plt.subplot(2,3,i + 4)
    plt.plot(data_resampled[label].value_counts())
    plt.xticks([])
    #plt.ylim([0,10000])
    plt.title(label.rstrip("DESCRIPTION").lstrip("MSSS"))
    ax.set_yscale('log')
    plt.ylim([1, 100000])

plt.tight_layout()

In [ ]:
data_train = data_resampled

# Export pre-processed dataset

In [ ]:
data_train.to_csv(
    os.path.join(EQ_DATA_PATH, "../data_train.csv"),
    index=False)

data_eval.to_csv(
    os.path.join(EQ_DATA_PATH, "../data_eval.csv"),
    index=False)